# TensorFlow 

O TensorFlow é uma biblioteca de computação numérica desenvolvida pelo Goolge e tornada *open source* em 2015, tornando-se rapidamente a mais famosa e recomendada para *Deep Learning*. No início de 2017, o TensorFlow saiu da sua versão beta e ganhou suporte para o Windows. Algumas das vantagens que fazem o TensorFlow se destacar das outras bibliotecas de redes neurais são;

* API principal em Python com implementação extremamente eficiente de computações em C++. Também suporta utilização de GPUs para maior paralelização dos processamentos;
* Muitas APIs de alto nível, como [Keras](https://keras.io/) e [TFLearn](http://tflearn.org/) que tornam a implementação de redes neurais extremamente simples;
* Permite treinar redes neurais em colossais com bilhões de dados. Isso não é nenhuma surpresa, considerando que o TensorFlow foi desenvolvido pelo Google justamente para esse propósito;
* Grande flexibilidade (ao custo de maior complexidade) no API original de Python, permitindo criar qualquer arquitetura de rede neural;
* Várias funções de otimização com implementação de diferenciação automática. Assim, não precisamos nos preocupar com os gradientes e podemos deixar essa tarefa para o TensorFlow;
* Uma ótima ferramenta de visualização chamada *TensorBoard*.

## Instalação

Se você está usando Linux, a instalação deve ser tão simples como `pip3 install tensorflow`. Contudo, isso pode variar de sistema operacional para sistema operacional. Além disso, a instalação pode ser mais complicada case se deseje utilizar suporte para GPU. Assim, eu prefiro recomendar a [referência oficial do TensorFlow para instalação](https://www.tensorflow.org/install/).

## Fluxograma TensorFlow

Um programa TensorFlow é dividido em duas partes: 

1) Montagem do modelo ou grafo.  
2) Execução do grafo

Na primeira parte, nenhuma computação de fato acontece. Nós só estruturamos uma série de operações que serão rodadas na segunda parte. A execução do grado é tipicamente um loop de treinamento, no qual a cada iteração rodamos o grafo montado na primeira parte. Para entender melhor como isso acontece, vamos considerar como exemplo a construção do algoritmo de regressão linear.

In [1]:
import numpy as np # para computação numérica menos intensivas
np.random.seed(42) # para consistência nos resultados

import os # para criar pastas

from sklearn.datasets import fetch_california_housing # base de dados
from sklearn.metrics import r2_score # para avalização

import tensorflow as tf
print(tf.__version__)

1.1.0


Acima, só realizamos algumas importações. Note como estamos utilizando a versão 1.1.0 do TensorFlow. Agora, vamos baixar os dados para usar no nosso exemplo. Essa base diz respeito ao preço de casas em bairros a Califórnia. Nossa variável de interesse é o preço médio do bairro (em 100'000 dólares).

In [2]:
X = fetch_california_housing().data # aqui pegamos as variáveis preditoras
y = fetch_california_housing().target # aqui pegamos a variável, independente

print('Formato dos dados:', X.shape, y.shape)

Formato dos dados: (20640, 8) (20640,)


### Fase de construção

Agora que temos os dados, vamos para **construção do grafo** no TensorFlow. Um grafo, aqui, deve ser entendido como nada mais do que uma sequência de operações computacionais. Para montar o grafo, nós vamos criar uma instância `tf.Graph()` e dentro dela vamos colocar as operações e variável que quereremos executar mais para frente. O que criamos dentro do grafo é normalmente chamado de *nodes* (nós) do grafo ou *ops* (operações).

Além disso, antes de montar o grafo eu gosto de definir algumas constantes que serão utilizadas tanto na montagem quanto na execução do grafo. Geralmente essas constantes são os hiper-parâmetros do modelo que montaremos e tê-los todos em um só lugar pode ajudar bastante no momento de ajustá-los. Particularmente, nesse caso vamos implementar uma regressão linear com gradiente descendente. Assim, precisamos definir o número de iterações de treino e a taxa de aprendizado. Para conveniência, eu também gosto de definir o formato dos dados. 

In [3]:
# definindo constantes
lr = 1e-2 # taxa de aprendizado
n_iter = 2501 # número de iterações de treino
n_inputs = X.shape[1] # número de variáveis independentes
n_outputs = 1 # número de variáveis dependentes

graph = tf.Graph() # isso cria um grafo
with graph.as_default(): # isso abre o grafo para que possamos colocar operações e variáveis dentro dele.
    tf.set_random_seed(1)
    
    # adiciona as variáveis ao grafo
    W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=.1), name='Weight')
    b = tf.Variable(tf.zeros([n_outputs]), name='bias')


    ######################################
    # Monta o modelo de regressão linear #
    ######################################

    # Camadas de Inputs
    x_input = tf.placeholder(tf.float32, [None, n_inputs], name='X_input')
    y_input = tf.placeholder(tf.float32, [None, n_outputs], name='y_input')

    # Camada Linear
    y_pred = tf.add(tf.matmul(x_input, W), b, name='y_pred')

    # Camada de custo ou função objetivo
    EQM = tf.reduce_mean(tf.square(y_pred - y_input), name="EQM")

    # otimizador
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(EQM)

    # inicializador
    init = tf.global_variables_initializer()

    # para salvar o modelo treinado
    saver = tf.train.Saver()


Vamos agora destrinchar esse código para entender o que está acontecendo. Primeiro, nós definimos algumas constantes. Não há nenhum segredo aqui e sequer utilizamos o TensorFlow para isso. Em seguida, criamos um grafo e abrimos ele com o comando `with`. Dentro do grafo, nós começamos definindo as variáveis do modelo de regressão linear. Você deve lembrar que ele é $\pmb{y}=\pmb{X}\pmb{w} + \pmb{b} + \pmb{\epsilon}$. Assim, as nossas variáveis são $\pmb{w}$ e $\pmb{b}$. Na nomenclatura de redes neurais, elas recebem os nomes de pesos e vieses, respectivamente. Aqui, nós vamos iniciar os pesos com pequenos valores aleatórios, segunda uma distribuição normal com média zero (padrão) e desvio padrão 0,1; o viés será inicializado em zero. Note como utilizamos a função `tf.Variable()` para definir as variáveis. Esse comando autoriza o TensorFlow a atualizar o valor numérico das variáveis durante o treinamento. Note também como passamos o formato das variáveis para sua construção.

Uma vez que tenhamos as variáveis, partimos para a construção do modelo. Em primeiro lugar, definimos a camada de *input* com a função `tf.placeholder()`. Esses *placeholders* são espaços que serão alimentados com os nossos dados. Por hora, eles não contêm nada, mas nós já definimos o formato deles. Por exemplo, o *placeholder* para as variáveis independentes têm o formato `[None, n_inputs]`, em que `n_inputs` é a quantidade de variáveis independentes e usamos `None` para dizer que a dimensão de quantidade de observações poderá variar.

A seguir, definimos a camada linear $\pmb{X}\pmb{w} + \pmb{b}$. Para isso, usamos `tf.matmul()` para multiplicação de matriz e `tf.add()` para adição. O resultado dessa operação será um vetor de previsões. Na camada de custo, nós vamos subtrair os $y$s reais dos $y$s previstos, elevar isso ao quadrado com `tf.square()` e tirar a média para produzir o erro quadrático médio. Isso finaliza o nosso modelo. Um detalhe é que podemos usar funções como `tf.add()` e o sinal `+` de maneira equivalente. Aqui, nós optamos por utilizar `tf.add()` para conseguir nomear a operação, o que é uma boa prática de organização do código.

Por fim, nós definimos um otimizador. Aqui, vamos  utilizar o otimizador Adam, que é uma versão melhorada do gradiente descendente; ele incorpora momento e taxa de aprendizado adaptativa. Independentemente do otimizador utilizado, ele toma conta tanto do cálculo do gradiente quanto de atualizar as variáveis. Note como passamos para ele tanto o que queremos minimizar (`EQM`) quanto a taxa de aprendizado.

As últimas operações do nosso grafo são um inicializador e um *saver*. O inicializador será utilizado para inicializar as variáveis que definimos mais acima. É importante ressaltar que até esse momento nenhuma computação foi realizada. Nem mesmo as variáveis foram inicializadas. Por fim, o *saver* faz o que se espera: salva **as variáveis** do modelo treinado. Isso é útil caso queiramos continuar o treinamento mais tarde ou realizar previsões depois. Termina assim nossa fase de construção do grafo.

### Fase de Execução

Na fase de execução, tipicamente realizamos iterações de execuções do grafo e reportamos alguma métrica de progresso, como o valor da função custo. Nós não executamos as computações do grafo em Python, mas em alguma linguagem mais eficiente. Pense no TensorFlow como sendo uma linguagem em si, que é executada em baixo nível, fora do Python. Assim, para interagir com o grafo TensorFlow, precisamos abrir de uma sessão TensorFlow. A execução do grafo será feita por meio dela.

In [4]:
# criamos uma pasta para salvar o modelo
if not os.path.exists('tmp'):
    os.makedirs('tmp')

# abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    sess.run(init) # iniciamos as variáveis
    
    # cria um feed_dict
    feed_dict = {x_input: X, y_input: y.reshape(-1,1)}
    
    # realizamos as iterações de treino
    for step in range(n_iter + 1):
        
        # executa algumas operações do grafo
        _, l = sess.run([optimizer, EQM], feed_dict=feed_dict)
        
        if (step % 500) == 0:
            print('Custo na iteração %d: %.2f \r' % (step, l), end='')
            saver.save(sess, "./tmp/my_model.ckpt")


Em primeiro lugar, nós criamos uma pasta `tmp/`, onde salvaremos as variáveis do nosso modelo. Depois, abrimos a sessão para conseguir interagir com o grafo TensorFlow. Note como passamos como argumento para a sessão o grafo ciado anteriormente: `tf.Session(graph=graph)`. Dentro da sessão, a primeira coisa que fazemos é inicializar as variáveis. A função `sess.run()` executa um nó do grafo e nesse caso estamos executando o nó de inicialização das variáveis. Em seguida, nós criamos um `feed_dict`, um dicionário que referencia quais dados serão passados para quais *placeholders*. Lembre-se de que temos dois: um para colocar as variáveis dependentes (`x_input`) e outro para colocar a variável independente (`y_input`). Nosso *placeholder* da variável $y$ é do formato `[None, 1]`, mas a nossa variável $y$ é um array numpy de uma única dimensão. Por isso, usamos `y.reshape(-1,1)` para transformá-lo em um array de duas dimensões `[n_obs, 1]`.

Depois disso, iniciamos o loop de treinamento. Esse é a parte mais importante da fase de execução. Nesse loop, vamos rodar as iterações de treinamento do modelo. Para isso, novamente usamos `sess.run()` só que dessa vez o primeiro argumento é uma lista de nós do grafo para rodarmos. Dentre esses argumentos, passamos a operação de treino `optimizer`. Essa operação minimiza a função `EQM`, e por isso depende dela, assim, embora só tenhamos passado `optimizer` como argumento para `sess.run()`, todas as operações necessárias para calcular `EQM` são computadas também, isto é, o próprio nó `EQM` e a multiplicação de matriz na camada linear. Como essa última depende dos valores nos *placeholders*, nós precisamos passar como argumento de `sess.run()` o `feed_dict` que mapeia quais dados serão colocados no *placeholder*. Um detalhe importante é que passamos para `sess.run()`, na lista de nós para rodar, `EQM`. Isso não é necessário para o treinamento, já que ao rodar `optimizer` a operação `EQM` já será rodada pelo fato de `optimizer` depender dela. No entanto, nós queremos trazer o valor numérico de `EQM` para o Python e reportá-lo a cada 500 iterações de treino. O fato de você não ver o que está sendo executado nos fundos pelo TensorFlow é algo que torna essa biblioteca particularmente contra intuitiva. Para entender o que está acontecendo, lembre que **ao usar `sess.run()` em um nó do grafo, todos os nós dos quais ele depende também serão executados**. No caso do nó `optimizer`, mais duas coisas acontecem: (1) os gradientes são calculados e (2) as variáveis são atualizada conforme a regra definida pelo otimizador.

Por fim, a última etapa da iteração de treino é um condicional que executa seu conteúdo a cada 500 iterações. Dentro dele, nós primeiro reportamos o custo daquela iteração e depois salvamos o modelo com o nome `my_model.ckpt` na pasta `tmp/`. Uma dúvida que pode surgir aqui é porque estamos calculando o valor da função custo a cada iteração se apenas o reportamos a cada 500 iterações? Isso não está deixando nosso treinamento ineficiente? A resposta é não, pois, ao executar `optimizer`, já estamos incorrendo no custo computacional de calcular `EQM` e o custo de alocá-lo para uma variável é mínimo. Um outro detalhe é que salvar o modelo no disco geralmente é uma operação demorada, então fazer isso com muita frequência tornará o treinamento mais lento. 

Após treinado, nós podemos restaurar o modelo salvo para fazer previsões. Novamente, isso tem que ser feito dentro de uma sessão TensorFlow. Agora, no entanto, em vez de inicializarmos as variáveis como tínhamos feito antes, nós restauramos o modelo treinado utilizando o *saver*.

In [5]:
# novamente, abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    
    # restauramos o valor das variáveis 
    saver.restore(sess, "./tmp/my_model.ckpt", )
    
    # rodamos o nó de previsão no grafo
    y_hat = sess.run(y_pred, feed_dict={x_input: X})
    
    print('\nR2: %.3f' % r2_score(y_pred=y_hat, y_true=y))


INFO:tensorflow:Restoring parameters from ./tmp/my_model.ckpt

R2: 0.499


Acima, após restaurar o modelo, nós rodamos o nó `y_pred`. Note como esse nó depende apenas do *placeholder* das variáveis independentes, por isso criamos um `feed_dict` mapeando apenas o que passaremos para o *placeholder* `x_input`. Por fim, nós reportamos o $R^2$ do modelo. Note que esse $R^2$ diz respeito aos dados de treinamento, mas como esse é um tutorial de TensorFlow e não de aprendizado de máquina não me dei ao trabalho de separar os dados em duas subamostras. Na prática, isso terá que ser feito. Sobre a restauração do modelo, é preciso fazer um **aviso importante**. O comportamento padrão de `saver.restore()` é restaurar todas as variáveis com os seus **nomes originais**. Isso significa que antes de restaurar o modelo, nós precisamos construir o grafo **mantendo os nomes das variáveis do modelo salvo!**. Por exemplo, rodar o código acima sem ter antes feito a fase de construção como fizemos resultaria em um `NotFoundError`. Para ver isso, experimente mudar apenas o nome de alguma variável na célula de construção (por exemplo, troque 'bias' por 'b'), execute aquela célula novamente para atualizar o nome da variável no grafo e então rode a célula que restaura o modelo para fazer as previsões sem treinar o modelo novamente.

## Considerações Finais

O que vimos até aqui aborda todo o conteúdo essencial do TensorFlow e com pequenas mudanças no código acima já será possível treinar redes neurais. Particularmente, nós vimos que o TensorFlow mesmo sendo feito principalmente para Python é quase uma linguagem de computação em si, com regras próprias e bem diferentes das do Python. E como toda linguagem de computação, no TensorFlow há varias formas de realizar a mesma tarefa. Acima, nós só abordamos uma, mas vale a pena destacar alguns comandos equivalentes bem comuns, para facilitar na leitura de código de outras pessoas. Começando pela criação do grafo.

### Criação do Grafo 

Eu disse que para criar um grafo TensorFlow era preciso utilizar os comandos

In [6]:
# fase de construção
some_graph = tf.Graph() # cria o grafo
with some_graph.as_default():
    [...] # adiciona nós ao grafo

# fase de execução
with tf.Session(graph=some_graph) as sess:
    [...] # executa o grafo

Isso não é de todo verdade. Embora recomendado, você pode criar o grafo sem usar `tf.Graph()` e o TensorFlow adicionará as operações/nós a um grafo padrão. Assim, as fases de construção seriam da seguinte forma

In [7]:
# fase de construção
[...] # criação o grafo

with tf.Session() as sess:
    [...]  # executa o grafo

A primeira forma é mais organizada, pois deixa bem clara a divisão entre as duas fases de um típico programa TensorFlow. Além disso, a primeira opção é útil quando estamos trabalhando com mais de um grafo. Mesmo assim, a segunda opção é menos formal e eu mesmo acabo optando por ela muitas vezes.

### Criação de Variáveis

Ao criar variáveis, nós usamos `tf.Variable()`. Outra opção seria utilizar `tf.get_variable()`, que cria uma variável nova, caso ela ainda não exista, ou reusa a variável, caso ela já exista. a funcionalidade reutilizar uma variável é mais complexa e não será discutida aqui, já que nosso objetivo é apenas criar variáveis. No entanto, é preciso dizer que simplesmente usar `tf.get_variable()` com uma variável que já existe resultará em um erro; isso é feito para evitar reutilizar variáveis sem querer.

A vantagem de utilizar `tf.get_variable()` é que podemos passar como argumento um inicializador. Isso pode não parecer importante agora, mas um dos fatores que possibilitou o recente sucesso das redes neurais é saber como inicializar os seus parâmetros e fazer isso na mão pode ser entendiante. Assim, utilizamos `tf.get_variable()` para criar variáveis da seguinte forma.

In [8]:
tf.reset_default_graph() # limpa o grafo de todas as variáveis
W = tf.get_variable('Weight', shape=[n_inputs, n_outputs],
                    initializer=tf.truncated_normal_initializer(stddev=0.1))

Para evitar o erro de utilizar `tf.get_variable()` com variáveis já existentes no grafo, nós primeiro limpamos o grafo de todas as variáveis. Note que `tf.reset_default_graph()` excluirá do grafo todas os nós adicionados ao grafo até aquele ponto, então tome cuidado para não utilizá-lo no meio da criação de um grafo! Note também como passamos para `tf.get_variable()` um inicializador. Nesse caso, usamos simplesmente `tf.truncated_normal_initializer` para iniciar a variável com pequenos valores aleatórios.

### Interagindo com o TensorFlow na sessão

Para interagir com o TensorFlow, utilizamos dentro da sessão `sess.run()` nas operações que queremos executar. Esse método é o mais geral e funciona tanto para acessar o valor de variáveis quanto para executar operações como a de treino. No entanto, além dele, temos outra forma de trazer o valor variáveis do TensorFlow para o Python. Essa outra opção é usar o método `.eval()` das próprias variáveis. Ainda assim, esse método **deve ser usado dentro de uma sessão** e não funciona com operações. Por exemplo, suponha que queiramos saber qual o valor do viés `b` do modelo treinado. Ou então que queremos saber qual foi o valor final da função custo. Isso pode ser feito da seguinte forma

In [9]:
# novamente, abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    
    # restauramos o modelo salvo
    saver.restore(sess, "./tmp/my_model.ckpt", )
    
    # duas formas de acessar o valor de variáveis
    print(sess.run(b))
    print(b.eval())
    
    # acessa o valor da função final
    print(EQM.eval(feed_dict={x_input: X, y_input: y.reshape(-1,1)}))
    

INFO:tensorflow:Restoring parameters from ./tmp/my_model.ckpt
[ 0.01486106]
[ 0.01486106]
0.667202


No caso de rodar `EQM` com `.eval()`, nós também precisamos passar um `feed_dict`, já que esse nó do grafo depende dos placeholders.

Com isso finalizamos esse tutorial de TensorFlow essencial

## Referências

O TensorFlow é extremamente recente e mesmo a documentação original do pacote não é muito boa. Mesmo assim, ela pode ser útil em alguns casos, principalmente os [tutoriais mais básicos](https://www.tensorflow.org/get_started/). Fora isso, eu recomendo a segunda parte do livro [Hands-On Machine Learning with Scikit-Learn and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291), que foi o conteúdo mais claro que li sobre o TensorFlow. 

Uma outra opção para tutoriais é procurar vídeos no YouTube. Quanto a isso, tenho duas playlists para recomendar

1) [TensorFlow, por Hvass Laboratories](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)  
2) [TensorFlow, por 周莫烦](https://www.youtube.com/playlist?list=PLXO45tsB95cJHXaDKpbwr5fC_CCYylw1f)

# Exercício

Use o TensorFlow para cria um modelo de regressão logística e prever se uma pessoa ganha mais de 50k. As quatro células seguintes baixam e processam os dados do censo americano de 1994.

Link com a resposta: https://www.tensorflow.org/tutorials/wide

In [10]:
import tempfile
import urllib
import pandas as pd

# cria uma pasta temporária para salvar os dados
train_file = tempfile.NamedTemporaryFile() 
test_file = tempfile.NamedTemporaryFile()

# baixa os dados
urllib.request.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
urllib.request.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# nome das colunas
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]

# le os dados
df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)

In [11]:
# cria os targets
y_train = (df_train['income_bracket'].values == '<=50K.').astype(int)
y_test = (df_test['income_bracket'].values == '<=50K.').astype(int)

# retira os targets dos dados
df_train.drop('income_bracket', axis=1, inplace=True)
df_test.drop('income_bracket', axis=1, inplace=True)

In [12]:
# Escalona as variáveis numéricas
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
df_train[numerical] = scaler.fit_transform(df_train[numerical])
df_test[numerical] = scaler.transform(df_test[numerical])

# converte as variáveis categóricas para dummies.
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

X_train = df_train.values
X_test = df_test.values

In [13]:
def shuffle_Xy(x, y):
    rng_state = np.random.get_state()
    np.random.shuffle(x)
    np.random.set_state(rng_state)
    np.random.shuffle(y)
    
# embaralha os dados de treino
shuffle_Xy(X_train, y_train)
    
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(32561, 108) (32561,)
(16281, 107) (16281,)


Primeiro, construa o grafo com o modelo de regressão logística. Eu sugiro tentar experimentar com métodos que eu não utilizei no tutorial, como `tf.get_variable()`. Tente também experimentar [outros otimizadores](https://www.tensorflow.org/api_guides/python/train).

In [14]:
# Defina as constantes
batch_size = 0
n_iter = 1
# ...

# Complete o grafo com o modelo de regressão linear
graph = tf.Graph()
with graph.as_default():
    pass
    
    # adicione variáveis ao grafo


    ######################################
    # Monte o modelo de regressão logística #
    ######################################

    # Camadas de Inputs

    # Camada Linear
    
    # Camada Logística
    
    # Camada de custo ou função objetivo

    # otimizador

    # inicializador

    # saver


Agora, execute o treinamento do modelo constriuido acima. 

In [15]:
# criamos uma pasta para salvar o modelo
if not os.path.exists('tmp'):
    os.makedirs('tmp')

# abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    
    # inicialize as variáveis
    
    # realizamos as iterações de treino
    for step in range(n_iter + 1):
        
        # Essa base de dados é mt grande para usar gradiente descendente
        # Assim, criamos mini-lotes para usar gradiente descendente estocástico
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size)]

        
        # executa operações de treino
        
        # acompanhe o treinamento com alguma estatísticas
        # salve o modelo de tempos em tempos
        if (step % 500) == 0:
            pass
    
    # agora que o modelo foi treinado, faça previsões
    
    
    # reporte a acurácia das suas precisões. Deve ser algo perto de 80%.
